In [1]:
import urllib.request as ur # libreria para manipular peticiones 
import re                   # libreria para manejar expresiones regulares

f = ur.urlopen('https://resultados.as.com/resultados/futbol/primera/jornada') # consulta a pagina web  
s = f.read().decode() # manipulacion de respuesta web 


In [2]:
# Método para eliminar tags html que no nos interesan con ER 
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr,'',raw_html)
    return cleantext

In [3]:
from pprint import pprint     # import para hacer prints boninitos 

#Obtener url de cada jornada 
urlJornadas = re.findall('<li>.*?cont-jornada" href="(.*?)".*?</li>',s,re.DOTALL)

In [4]:
import datetime # libreria para manejo de fecha 

now = datetime.datetime.now() #obtener fecha actual 
# Arreglo de meses que inicia en agosto como la temporada 
months = ['Ago','Sep','Oct','Nov','Dic','Ene','Feb','Mar','Abr','May','Jun','Jul'] 
day =  now.day # obtener el numero del dia actual 
month = (now.month + 4) % 12 # obtener mes actual y ajustarlo para que coincida con el orden de meses de la temporada

In [5]:
# print results 
def printResults():
    print("Resultados condensados")
    print("G:Ganados   P:Perdidos   E:Empatados   GF:Goles a favor   GC:Goles en contra")
    print("\nEquipo\t\tG\tP\tE\tGF\tGC")
    for x in tablaResultados:
        data = x
        if len(data) > 7:
            data = data + "\t" + str(tablaResultados.get(x)[0])
        else: 
            data = data + "\t\t" + str(tablaResultados.get(x)[0])
        data = data + "\t" + str(tablaResultados.get(x)[1])
        data = data + "\t" + str(tablaResultados.get(x)[2])
        data = data + "\t" + str(tablaResultados.get(x)[3])
        data = data + "\t" + str(tablaResultados.get(x)[4])
        print(data)

In [6]:
'''
Método goles a favor
'''
def aFavor(E,G):
    tablaResultados.get(E)[3] = tablaResultados.get(E)[3] + int(G)
    

In [7]:
'''
Método goles en contra
'''
def enContra(E,G):
    tablaResultados.get(E)[4] = tablaResultados.get(E)[4] + int(G)

In [8]:
'''
Método empatar
'''

def empatar(E1,E2,R1):
    tablaResultados.get(E1)[2] = tablaResultados.get(E1)[2] + 1
    tablaResultados.get(E2)[2] = tablaResultados.get(E2)[2] + 1
    aFavor(E1,R1)
    enContra(E1,R1)
    aFavor(E2,R1)
    enContra(E2,R1)

In [9]:
'''
Metodo ganar 
'''

def ganar(E,GF,GC): #E: equipo GF goles favor GC goles contra
    tablaResultados.get(E)[0] = tablaResultados.get(E)[0] + 1
    aFavor(E,GF)
    enContra(E,GC)
    
    

In [10]:
'''
Metodo perder
'''
def perder(E,GF,GC): #E: equipo GF goles favor GC goles contra
    tablaResultados.get(E)[1] = tablaResultados.get(E)[1] + 1
    aFavor(E,GF)
    enContra(E,GC)
    

In [11]:
'''
Lógica perder ganar o empatar
'''

def decideGanador(datos):
    if int(datos[2]) == int(datos[3]): #empate
        empatar(datos[0],datos[4],datos[2])
    elif int(datos[2]) > int(datos[3]): # gana equipo1 
        ganar(datos[0],datos[2],datos[3])
        perder(datos[4],datos[3],datos[2])
    else: # gana equipo 2
        ganar(datos[4],datos[3],datos[2])
        perder(datos[0],datos[2],datos[3])

In [12]:
'''
Tabla que contiene los resultados 
Los datos tienen la siguiente estructura 
{'Equipo':['Ganados','Perdidos','Empatados','Goles Favor','Goles Contra'],...}
'''

tablaResultados = {}
def addResults(listaResultados):
    for x in listaResultados:
        if(tablaResultados.get(x[0]) is None):
            tablaResultados.update({x[0]:[0,0,0,0,0]})
        if(tablaResultados.get(x[4]) is None):
            tablaResultados.update({x[4]:[0,0,0,0,0]})
        decideGanador(x)

In [13]:
'''
Métodos para lógica del pasado 
'''
# Para pasados 
def pasados():
    # Obtener Equipo1 GolesE1 GolesE2 Equipo2
    marcadorPasado = re.findall('class="nombre-equipo".*?>(.*?)</span>.*?class="resultado">.*?((..).-.(..)).*?</div>.*?class="nombre-equipo".*?>(.*?)</span>',jornadai,re.DOTALL)
    print("Resultados: ")
    addResults(marcadorPasado)
    print()
    for x in marcadorPasado:
        print(x[0] + x[1] +x[4])

def printPast(jornadaFecha):
    if jornadaFecha[0][2] == '':  
        if jornadaFecha[0][3] != '':
            print("La " + jornadaFecha[0][0] + " se realizó del " + jornadaFecha[0][1] + " al "+ jornadaFecha[0][3] + " de " + jornadaFecha[0][4] )
        else: 
             print("La " + jornadaFecha[0][0] + " se realizó el " + jornadaFecha[0][1] + " de " + jornadaFecha[0][4] )
    else:
        if jornadaFecha[0][3] != '':
            print("La " + jornadaFecha[0][0] + " se realizó del " + jornadaFecha[0][1] + " de " + jornadaFecha[0][2] + " al "+ jornadaFecha[0][3] + " de " + jornadaFecha[0][4] )
        else: 
             print("La " + jornadaFecha[0][0] + " se realizó el " + jornadaFecha[0][1] + " de " + jornadaFecha[0][4] )
    pasados()

    


In [14]:
'''
Métodos para lógica de actual 
'''

# Para actuales 
# Obtener Equipo1 GolesE1-GolesE2 GolesE1 GolesE2 Equipo2 para los que lo tengan 
def actuales():
    listaResultados = jornadai.split('<li class="list-resultado">')
    listaResultados.pop(0)

    jugadosActual = []
    futurosActual = []

    for x in range(10):
        marcadorActual = []
        marcadorActual = re.findall('class="nombre-equipo".*?>(.*?)</span>.*?class="resultado">.*?((..).-.(..)).*?</div>.*?class="nombre-equipo".*?>(.*?)</span>',listaResultados[x],re.DOTALL)
        if marcadorActual[0][1] == '   -   ': 
            juegosFuturos = re.findall('class="nombre-equipo".*?>(.*?)</span>.*?class="nombre-equipo".*?>(.*?)</span>.*?<span class="fecha">(.*?)</span>',listaResultados[x],re.DOTALL)
            futurosActual.append(juegosFuturos[0])
        else:
            jugadosActual.append(marcadorActual[0])

    print("Resultados partidos jugados:")        
    for x in jugadosActual: 
        print(x[0] + x[1] +x[4])
    
    print("\nPartidos restantes:")
    for x in futurosActual:
        print(x[0] + " - " + x[1] + " en la fecha y horario " + x[2])

        

def printCurrent(jornadaFecha):
    print("La " + jornadaFecha[0][0] + " se está realizando")
    actuales()


In [15]:
'''
Métodos para lógica del futuro
'''

# para los que no lo tengan, obtener Equipo1 Equipo2 FechaDeJuego

# Para futuros
# obtener Equipo1 Equipo2 FechaDeJuego
def futuros():
    juegosFuturos = re.findall('class="nombre-equipo".*?>(.*?)</span>.*?class="nombre-equipo".*?>(.*?)</span>.*?<span class="fecha">(.*?)</span>',jornadai,re.DOTALL)
    print("Partidos pendientes:")
    for x in juegosFuturos:
        print(x[0] + " - " + x[1] + " en la fecha y horario " + x[2])

        
        
def printFuture(jornadaFecha): 
    if jornadaFecha[0][2] == '':
        if jornadaFecha[0][3] != '':
            print("La " + jornadaFecha[0][0] + " se realizará del " + jornadaFecha[0][1] + " al "+ jornadaFecha[0][3] +" de " + jornadaFecha[0][4] )
        else: 
            print("La " + jornadaFecha[0][0] + " se realizará el " + jornadaFecha[0][1] + " de " + jornadaFecha[0][4] )
            
    else: 
        if jornadaFecha[0][3] != '':
            print("La " + jornadaFecha[0][0] + " se realizará del " + jornadaFecha[0][1] + " de " + jornadaFecha[0][2]+ " al "+ jornadaFecha[0][3] +" de " + jornadaFecha[0][4] )
        else: 
            print("La " + jornadaFecha[0][0] + " se realizará el " + jornadaFecha[0][1] + " de " + jornadaFecha[0][4] )
    futuros()

In [16]:
'''Metodo para decidir punto en el tiempo'''

def pointInTime(jornadaFecha):
    if jornadaFecha[0][2] == '':  
        if months.index(jornadaFecha[0][4]) > month:
            printFuture(jornadaFecha)
        elif months.index(jornadaFecha[0][4]) < month:
            printPast(jornadaFecha)
        elif months.index(jornadaFecha[0][4]) == month:
            if int(jornadaFecha[0][1]) > day:
                printFuture(jornadaFecha)
            elif jornadaFecha[0][3] != '':
                if day <= int(jornadaFecha[0][3]) and  day >= int(jornadaFecha[0][1]):
                    printCurrent(jornadaFecha)
                else: 
                    printPast(jornadaFecha)
            elif int(jornadaFecha[0][1]) == day:
                printCurrent(jornadaFecha)
            else:
                printPast(jornadaFecha)
    else: 
        if months.index(jornadaFecha[0][2]) > month:
            printFuture(jornadaFecha)
        elif months.index(jornadaFecha[0][4]) < month:
            printPast(jornadaFecha)
        elif months.index(jornadaFecha[0][2]) < month: # estamos en mes 2 
            if jornadaFecha[0][3] < day:
                printPast(jornadaFecha)
            else:
                printCurrent(jornadaFecha)
        else: # Estamos en mes 1 
            if jornadaFecha[0][1] > day:
                printFuture(jornadaFecha)
            else:
                printCurrent(jornadaFecha)

In [17]:
#Abrir la jornada correspondiente para cada url 
for x in urlJornadas:
    f = ur.urlopen("https://resultados.as.com"+x)
    jornadai = f.read().decode()
    #print(s + "\n")
    #Obtener número de jornada y fecha 
    jornadaFecha = re.findall('<h2 class="tit-modulo">.*?(Jornada \d*).*?>(\d\d) (.*?)* *-* *(\d\d)* *(...)<.*?</h2>',jornadai,re.DOTALL)
    
    # Decidir si es pasado futuro o presente e imprimir datos
    pointInTime(jornadaFecha)

        
    print("\n\n\n")
    


La Jornada 1 se realizó del 17 al 20 de Ago
Resultados: 

Girona 0 - 0 Valladolid
Betis 0 - 3 Levante
Celta 1 - 1 Espanyol
Villarreal 1 - 2 R. Sociedad
Barcelona 3 - 0 Alavés
Eibar 1 - 2 Huesca
Rayo 1 - 4 Sevilla
Real Madrid 2 - 0 Getafe
Valencia 1 - 1 Atlético
Athletic 2 - 1 Leganés




La Jornada 2 se realizó del 24 al 27 de Ago
Resultados: 

Getafe 2 - 0 Eibar
Leganés 2 - 2 R. Sociedad
Alavés 0 - 0 Betis
Atlético 1 - 0 Rayo
Valladolid 0 - 1 Barcelona
Espanyol 2 - 0 Valencia
Sevilla 0 - 0 Villarreal
Girona 1 - 4 Real Madrid
Levante 1 - 2 Celta
Athletic 2 - 2 Huesca




La Jornada 3 se realizó del 31 al 24 de Oct
Resultados: 

Getafe 0 - 0 Valladolid
Villarreal 0 - 1 Girona
Eibar 2 - 1 R. Sociedad
Celta 2 - 0 Atlético
Real Madrid 4 - 1 Leganés
Levante 2 - 2 Valencia
Alavés 2 - 1 Espanyol
Barcelona 8 - 2 Huesca
Betis 1 - 0 Sevilla
Rayo 1 - 1 Athletic




La Jornada 4 se realizó del 14 al 17 de Sep
Resultados: 

Huesca 0 - 1 Rayo
Atlético 1 - 1 Eibar
R. Sociedad 1 - 2 Barcelona
Valencia

La Jornada 29 se realizará del 29 al 31 de Mar
Partidos pendientes:
Girona - Athletic en la fecha y horario V-29/03 21:00
Getafe - Leganés en la fecha y horario S-30/03 13:00
Barcelona - Espanyol en la fecha y horario S-30/03 16:15
Celta - Villarreal en la fecha y horario S-30/03 18:30
Alavés - Atlético en la fecha y horario S-30/03 20:45
Levante - Eibar en la fecha y horario D-31/03 12:00
Rayo - Betis en la fecha y horario D-31/03 14:00
Sevilla - Valencia en la fecha y horario D-31/03 16:15
Real Madrid - Huesca en la fecha y horario D-31/03 18:30
Valladolid - R. Sociedad en la fecha y horario D-31/03 20:45




La Jornada 30 se realizará el 03 de Abr
Partidos pendientes:
Sevilla - Alavés en la fecha y horario X-03/04 18:30
Valencia - Real Madrid en la fecha y horario X-03/04 18:30
Villarreal - Barcelona en la fecha y horario X-03/04 18:30
R. Sociedad - Betis en la fecha y horario X-03/04 18:30
Leganés - Valladolid en la fecha y horario X-03/04 18:30
Athletic - Levante en la fecha y hor

In [18]:
printResults()

Resultados condensados
G:Ganados   P:Perdidos   E:Empatados   GF:Goles a favor   GC:Goles en contra

Equipo		G	P	E	GF	GC
Girona		7	9	10	27	34
Valladolid	6	12	8	20	34
Betis		10	10	6	30	32
Levante		8	12	6	37	46
Celta		6	13	7	36	44
Espanyol	9	11	6	31	39
Villarreal	4	11	11	27	35
R. Sociedad	9	9	8	30	27
Barcelona	18	2	6	66	25
Alavés		11	8	7	27	29
Eibar		8	8	10	34	35
Huesca		5	14	7	26	42
Rayo		6	15	5	28	45
Sevilla		10	9	7	41	34
Real Madrid	15	8	3	43	31
Getafe		11	6	9	34	23
Valencia	7	4	15	27	21
Atlético	15	3	8	38	17
Athletic	7	7	12	25	30
Leganés		8	9	9	27	31
